In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [ ]:
import imbrium
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

imbrium.__version__

In [ ]:
# delete files in:  C:\Users\<UserName>\AppData\Local\Temp\.tensorboard-info
#%load_ext tensorboard

In [ ]:
from imbrium import PureUni

In [ ]:
example_data = pd.read_csv('example_dataset/AirQualityUCI.csv', delimiter=';')
example_data.head()

In [ ]:
example_data = example_data[['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)']]

In [ ]:
example_data.isna().sum()

In [ ]:
example_data = example_data.dropna()

In [ ]:
example_data.notna().sum()

In [ ]:
example_data = example_data.replace(",", ".", regex = True).astype("float")

In [ ]:
uni_var_data_example = pd.DataFrame(example_data['PT08.S1(CO)'])

In [ ]:
uni_var_data_example.plot()

In [ ]:
scaler = StandardScaler()

# Fit and transform the data using the scaler
normalized_uni_var_data_example = pd.DataFrame(scaler.fit_transform(uni_var_data_example))

In [ ]:
normalized_uni_var_data_example.plot()

In [ ]:
normalized_uni_var_data_example = np.array(normalized_uni_var_data_example)
normalized_uni_var_data_example

## UnivarMultiStep

In [ ]:
layer_config = {
   "layer0": {"config": {"neurons": 300, "activation": "relu", "regularization": 0.01, "dropout": 0.2}},
   "layer1": {"config": {"neurons": 150, "activation": "relu", "regularization": 0.01, "dropout": 0.2}},
   "layer2": {"config": {"neurons": 100, "activation": "relu", "regularization": 0.0}},

}

In [ ]:
predictor1 = PureUni(steps_past =  100, steps_future = 5, target = normalized_uni_var_data_example)

predictor1.create_mlp(
                      optimizer= 'adam',
                      loss= 'mean_squared_error',
                      metrics = 'mean_squared_error',
                      dense_block_one = 1,
                      dense_block_two = 1,
                      dense_block_three = 1,
                      layer_config = layer_config
                     )

In [ ]:
predictor1.fit_model(
                     epochs = 5,
                     show_progress= 1,
                     validation_split = 0.20,
                     board = False
                    )

In [ ]:
predictor1.show_performance().history

In [ ]:
predictor1.predict(normalized_uni_var_data_example[-100:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_mlp(epochs=1)

In [ ]:
predictor2 = PureUni(steps_past =  5, steps_future = 10, target = normalized_uni_var_data_example)

predictor2.create_rnn(
                      optimizer = 'adam', 
                      loss = 'mean_squared_error', 
                      metrics = 'mean_squared_error',
                      layer_config = layer_config
                     )

predictor2.fit_model(
                     epochs = 7,
                     show_progress= 1,
                     validation_split = 0.20,
                     monitor='val_loss',
                     patience=1,
                     min_delta = 0,
                     verbose = 1
                    )

In [ ]:
predictor2.show_performance()

In [ ]:
predictor2.predict(normalized_uni_var_data_example[-5:])

In [ ]:
predictor2.model_blueprint()

In [ ]:
predictor2.create_fit_rnn(epochs=1)

In [ ]:
predictor3 = PureUni(steps_past =  5, steps_future = 10, target = normalized_uni_var_data_example)

predictor3.create_lstm(
                       optimizer = 'adam',
                       loss = 'mean_squared_error',
                       metrics = 'mean_squared_error',
                       layer_config = layer_config
                      )

predictor3.fit_model(
                     epochs = 5,
                     show_progress= 1,
                     validation_split = 0.20
                    )

In [ ]:
predictor3.show_performance()

In [ ]:
predictor3.predict(normalized_uni_var_data_example[-5:])

In [ ]:
predictor3.model_blueprint()

In [ ]:
predictor3.show_performance()

In [ ]:
predictor3.create_fit_lstm(epochs=1)

In [ ]:
predictor4 = PureUni(steps_past =  5, steps_future = 10, target = normalized_uni_var_data_example)

predictor4.create_gru(
                      optimizer = 'adam',
                      loss = 'mean_squared_error',
                      metrics = 'mean_squared_error',
                      layer_config = layer_config
                     )
predictor4.fit_model(
                     epochs = 2,
                     show_progress= 1,
                     validation_split = 0.20
                    )

In [ ]:
predictor4.show_performance()

In [ ]:
predictor4.predict(normalized_uni_var_data_example[-5:])

In [ ]:
predictor4.model_blueprint()

In [ ]:
predictor4.create_fit_gru(epochs=1)

In [ ]:
layer_config_cnn = {
   "layer0": {"config": {"filters": 64, "kernel_size": 2, "activation": "relu", "regularization": 0.0, "dropout": 0.0}},
   "layer1": {"config": {"filters": 100, "kernel_size": 2, "activation": "relu", "regularization": 0.0, "dropout": 0.0}},
   "layer2": {"config": {"pool_size": 4}},
   "layer3": {"config": {"neurons": 100, "activation": "relu", "regularization": 0.0}},

}

In [ ]:
predictor5 = PureUni(steps_past =  16, steps_future = 10, target = normalized_uni_var_data_example)

predictor5.create_cnn(
                      optimizer = 'adam',
                      loss = 'mean_squared_error',
                      metrics = 'mean_squared_error',
                      layer_config = layer_config_cnn
                     )

predictor5.fit_model(
                     epochs = 2,
                     show_progress= 1,
                     validation_split = 0.20
                    )

In [ ]:
predictor5.show_performance()

In [ ]:
predictor5.predict(normalized_uni_var_data_example[-16:])

In [ ]:
predictor5.model_blueprint()

In [ ]:
predictor5.create_fit_cnn(epochs=1)

In [ ]:
layer_config_bi = {
   "layer0": {"config": {"neurons": 50, "activation": "relu", "regularization": 0.0, "dropout": 0.0}},
   "layer1": {"config": {"neurons": 50, "activation": "relu", "regularization": 0.0}},
}

In [ ]:
predictor6 = PureUni(steps_past =  5, steps_future = 10, target = normalized_uni_var_data_example)

predictor6.create_birnn(
                        optimizer = 'adam',
                        loss = 'mean_squared_error',
                        metrics = 'mean_squared_error',
                        layer_config = layer_config_bi
                       )
predictor6.fit_model(
                     epochs = 2,
                     show_progress= 1, 
                     validation_split = 0.20
                    )

In [ ]:
predictor6.show_performance()

In [ ]:
predictor6.predict(normalized_uni_var_data_example[-5:])

In [ ]:
predictor6.model_blueprint()

In [ ]:
predictor6.create_fit_birnn(epochs=1)

In [ ]:
predictor7 = PureUni(steps_past =  5, steps_future = 10, target = normalized_uni_var_data_example)

predictor7.create_bilstm(
                         optimizer = 'adam',
                         loss = 'mean_squared_error',
                         metrics = 'mean_squared_error',
                         layer_config = layer_config_bi
                        )
predictor7.fit_model(
                     epochs = 7,
                     show_progress= 1,
                     validation_split = 0.20,
                     monitor='val_loss',
                     patience=1,
                     min_delta = 0,
                     verbose = 1
                    )

In [ ]:
predictor7.show_performance()

In [ ]:
predictor7.predict(normalized_uni_var_data_example[-5:])

In [ ]:
predictor7.model_blueprint()

In [ ]:
predictor7.create_fit_bilstm(epochs=1)

In [ ]:
predictor8 = PureUni(steps_past =  5, steps_future = 10, target = normalized_uni_var_data_example)

predictor8.create_bigru(
                        optimizer = 'adam',
                        loss = 'mean_squared_error',
                        metrics = 'mean_squared_error',
                        layer_config = layer_config_bi
                       )
predictor8.fit_model(
                     epochs = 2,
                     show_progress= 1,
                     validation_split = 0.20
                    )

In [ ]:
predictor8.show_performance()

In [ ]:
predictor8.predict(normalized_uni_var_data_example[-5:])

In [ ]:
predictor8.model_blueprint()

In [ ]:
predictor8.create_fit_bigru(epochs=1)

In [ ]:
predictor8.freeze("test1.keras")

In [ ]:
load = PureUni(steps_past =  5, steps_future = 10)

In [ ]:
load.set_model_id('Loaded Model')

In [ ]:
load.retrieve('test1.keras')

In [ ]:
load.predict(normalized_uni_var_data_example[-5:])

### HybridUni

In [ ]:
from imbrium import HybridUni

In [ ]:
layer_config_cnn_hybrid = {
            "layer0": {
                "config": {
                    "filters": 64,
                    "kernel_size": 1,
                    "activation": "relu",
                    "regularization": 0.0,
                    "dropout": 0.0,
                }
            },
            "layer1": {
                "config": {
                    "filters": 32,
                    "kernel_size": 1,
                    "activation": "relu",
                    "regularization": 0.0,
                    "dropout": 0.0,
                }
            },
            "layer2": {
                "config": {
                    "pool_size": 2,
                }
            },
            "layer3": {
                "config": {
                    "neurons": 32,
                    "activation": "relu",
                    "regularization": 0.0,
                    "dropout": 0.0,
                }
            },
            "layer4": {
                "config": {
                    "neurons": 32,
                    "activation": "relu",
                    "regularization": 0.0,
                }
            },
        }

In [ ]:
predictor1 = HybridUni(sub_seq = 2, steps_past = 10, steps_future = 5, target = normalized_uni_var_data_example)

predictor1.create_cnnrnn(
                         optimizer= 'adam',
                         loss= 'mean_squared_error',
                         metrics = 'mean_squared_error', 
                         conv_block_one = 1,
                         conv_block_two = 1,
                         rnn_block_one = 1,
                         rnn_block_two = 1,
                         layer_config = layer_config_cnn_hybrid
                        )
predictor1.fit_model(
                     epochs = 3,
                     show_progress= 1,
                     validation_split = 0.20,
                     board = True
                     )

In [ ]:
predictor1.show_performance().history

In [ ]:
predictor1.predict(normalized_uni_var_data_example[-10:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_cnnrnn(epochs=1)

In [ ]:
predictor1.show_performance().history

In [ ]:
predictor1 = HybridUni(sub_seq = 8, steps_past = 16, steps_future = 8, target = normalized_uni_var_data_example)

predictor1.create_cnnlstm(
                          optimizer= 'adam',
                          loss= 'mean_squared_error',
                          metrics = 'mean_squared_error',
                          layer_config = layer_config_cnn_hybrid
                         )
predictor1.fit_model(
                     epochs = 7,
                     show_progress= 1,
                     validation_split = 0.20,
                     monitor='val_loss',
                     patience=1,
                     min_delta = 0,
                     verbose = 1
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(normalized_uni_var_data_example[-16:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_cnnlstm(epochs=1)

In [ ]:
predictor1 = HybridUni(sub_seq = 2, steps_past = 10, steps_future = 5, target = normalized_uni_var_data_example)

predictor1.create_cnngru(
                         optimizer= 'adam',
                         loss= 'mean_squared_error',
                         metrics = 'mean_squared_error',
                         layer_config = layer_config_cnn_hybrid
                        )
predictor1.fit_model(
                     epochs = 5,
                     show_progress= 1,
                     validation_split = 0.20
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(normalized_uni_var_data_example[-10:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_cnngru(epochs=1)

In [ ]:
layer_config_bicnn_hybrid = {
            "layer0": {
                "config": {
                    "filters": 64,
                    "kernel_size": 1,
                    "activation": "relu",
                    "regularization": 0.0,
                    "dropout": 0.0,
                }
            },
            "layer1": {
                "config": {
                    "filters": 32,
                    "kernel_size": 1,
                    "activation": "relu",
                    "regularization": 0.0,
                    "dropout": 0.0,
                }
            },
            "layer2": {
                "config": {
                    "pool_size": 2,
                }
            },
            "layer3": {
                "config": {
                    "neurons": 32,
                    "activation": "relu",
                    "regularization": 0.0,
                    "dropout": 0.0,
                }
            },
            "layer4": {
                "config": {
                    "neurons": 32,
                    "activation": "relu",
                    "regularization": 0.0,
                }
            },
        }

In [ ]:
predictor1 = HybridUni(sub_seq = 2, steps_past = 10, steps_future = 5, target = normalized_uni_var_data_example)

predictor1.create_cnnbirnn(
                           optimizer= 'adam',
                           loss= 'mean_squared_error',
                           metrics = 'mean_squared_error',
                           layer_config =  layer_config_bicnn_hybrid
                          )
predictor1.fit_model(
                     epochs = 2,
                     show_progress= 1,
                     validation_split = 0.20
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(normalized_uni_var_data_example[-10:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_cnnbirnn(epochs=1)

In [ ]:
predictor1 = HybridUni(sub_seq = 2, steps_past = 10, steps_future = 5, target = normalized_uni_var_data_example)

predictor1.create_cnnbilstm(
                            optimizer= 'adam',
                            loss= 'mean_squared_error',
                            metrics = 'mean_squared_error',
                            layer_config = layer_config_bicnn_hybrid
                           )
predictor1.fit_model(
                     epochs = 2,
                     show_progress= 1,
                     validation_split = 0.20
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(normalized_uni_var_data_example[-10:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_cnnbilstm(epochs=1)

In [ ]:
predictor1 = HybridUni(sub_seq = 2, steps_past = 10, steps_future = 5, target = normalized_uni_var_data_example)

predictor1.create_cnnbigru(
                           optimizer= 'adam',
                           loss= 'mean_squared_error',
                           metrics = 'mean_squared_error',
                           layer_config = layer_config_bicnn_hybrid
                          )
predictor1.fit_model(
                     epochs = 7,
                     show_progress= 1,
                     validation_split = 0.20,
                     monitor='val_loss',
                     patience=1,
                     min_delta = 0,
                     verbose = 1
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(normalized_uni_var_data_example[-10:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_cnnbigru(epochs=1)

In [ ]:
predictor1.freeze("test2.keras")

In [ ]:
load = HybridUni(sub_seq = 2, steps_past =  10, steps_future = 5)

In [ ]:
load.set_model_id('Loaded Model')

In [ ]:
load.retrieve('test2.keras')

In [ ]:
load.predict(normalized_uni_var_data_example[-10:])

## MultiVarMultiStep

In [ ]:
from imbrium import PureMulti

In [ ]:
target = np.array(example_data['PT08.S1(CO)']).reshape(-1, 1)

In [ ]:
target

In [ ]:
features = example_data[['CO(GT)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)']]

In [ ]:
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

# Scale the features and target separately
scaled_features = feature_scaler.fit_transform(features)
scaled_target = target_scaler.fit_transform(target)

In [ ]:
layer_config_multi =  {
            "layer0": {
                "config": {
                    "neurons": 50,
                    "activation": "relu",
                    "regularization": 0.0,
                    "dropout": 0.0,
                }
            },
            "layer1": {
                "config": {
                    "neurons": 50,
                    "activation": "relu",
                    "regularization": 0.0,
                    "dropout": 0.0,
                }
            },
            "layer2": {
                "config": {"neurons": 50, "activation": "relu", "regularization": 0.0}
            },
        }

In [ ]:
predictor1 = PureMulti(steps_past =  5, steps_future = 10, target = scaled_target, features = scaled_features)

predictor1.create_mlp(
                      optimizer= 'adam',
                      loss= 'mean_squared_error',
                      metrics = 'mean_squared_error',
                      layer_config = layer_config_multi
                     )
predictor1.fit_model(
                     epochs = 7,
                     show_progress= 1,
                     validation_split = 0.20,
                     monitor='val_loss',
                     patience=1,
                     min_delta = 0,
                     verbose = 1,
                     board = True
                    )

In [ ]:
predictor1.show_performance().history

In [ ]:
predictor1.predict(scaled_features[-5:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_mlp(epochs =3)

In [ ]:
predictor1.predict(scaled_features[-5:])

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1 = PureMulti(steps_past =  5, steps_future = 10, target = scaled_target, features = scaled_features)

predictor1.create_rnn(
                      optimizer= 'adam',
                      loss= 'mean_squared_error',
                      metrics = 'mean_squared_error',
                      layer_config = layer_config_multi
                     )
predictor1.fit_model(epochs = 10,
                     show_progress= 1,
                     validation_split = 0.20
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(scaled_features[-5:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_rnn(epochs = 1)

In [ ]:
predictor1 = PureMulti(steps_past =  5, steps_future = 10, target = scaled_target, features = scaled_features)

predictor1.create_lstm(
                       optimizer= 'adam',
                       loss= 'mean_squared_error',
                       metrics = 'mean_squared_error',
                       layer_config = layer_config_multi
                      )
predictor1.fit_model(
                     epochs = 7,
                     show_progress= 1,
                     validation_split = 0.20,
                     monitor='val_loss',
                     patience=1,
                     min_delta = 0,
                     verbose = 1
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(scaled_features[-5:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_lstm(epochs = 1)

In [ ]:
layer_config_multi_cnn = {
            "layer0": {
                "config": {
                    "filters": 64,
                    "kernel_size": 1,
                    "activation": "relu",
                    "regularization": 0.0,
                    "dropout": 0.0,
                }
            },
            "layer1": {
                "config": {
                    "filters": 32,
                    "kernel_size": 1,
                    "activation": "relu",
                    "regularization": 0.0,
                    "dropout": 0.0,
                }
            },
            "layer2": {
                "config": {
                    "pool_size": 2,
                }
            },
            "layer3": {
                "config": {
                    "neurons": 32,
                    "activation": "relu",
                    "regularization": 0.0,
                }
            },
        }

In [ ]:
predictor1 = PureMulti(steps_past =  5, steps_future = 10, target = scaled_target, features = scaled_features)

predictor1.create_cnn(
                      optimizer= 'adam',
                      loss= 'mean_squared_error',
                      metrics = 'mean_squared_error',
                      layer_config = layer_config_multi_cnn
                     )
predictor1.fit_model(
                     epochs = 2,
                     show_progress= 1,
                     validation_split = 0.20
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(scaled_features[-5:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_cnn(epochs = 1)

In [ ]:
predictor1 = PureMulti(steps_past =  5, steps_future = 10, target = scaled_target, features = scaled_features)

predictor1.create_gru(optimizer= 'adam',
                      loss= 'mean_squared_error',
                      metrics = 'mean_squared_error', 
                      layer_config = layer_config_multi
                     )
predictor1.fit_model(epochs = 2,
                     show_progress= 1,
                     validation_split = 0.20
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(scaled_features[-5:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_gru(epochs = 1)

In [ ]:
layer_config_bi_multi = {
            "layer0": {
                "config": {
                    "neurons": 50,
                    "activation": "relu",
                    "regularization": 0.0,
                    "dropout": 0.0,
                }
            },
            "layer1": {
                "config": {
                    "neurons": 50,
                    "activation": "relu",
                    "regularization": 0.0,
                }
            },
        }

In [ ]:
predictor1 = PureMulti(steps_past =  5, steps_future = 10, target = scaled_target, features = scaled_features)

predictor1.create_birnn(
                        optimizer= 'adam',
                        loss= 'mean_squared_error',
                        metrics = 'mean_squared_error',
                        layer_config = layer_config_bi_multi
                       )
predictor1.fit_model(
                     epochs = 2,
                     show_progress= 1,
                     validation_split = 0.20
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(scaled_features[-5:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_birnn(epochs = 1)

In [ ]:
predictor1 = PureMulti(steps_past =  3, steps_future = 10, target = scaled_target, features = scaled_features)

predictor1.create_bilstm(
                         optimizer= 'adam',
                         loss= 'mean_squared_error',
                         metrics = 'mean_squared_error',
                         layer_config = layer_config_bi_multi
                        )
predictor1.fit_model(
                     epochs = 2,
                     show_progress= 1,
                     validation_split = 0.20
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(scaled_features[-3:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_bilstm(epochs = 1)

In [ ]:
predictor1 = PureMulti(steps_past =  5, steps_future = 10, target = scaled_target, features = scaled_features)

predictor1.create_bigru(
                        optimizer= 'adam',
                        loss= 'mean_squared_error',
                        metrics = 'mean_squared_error',
                        layer_config = layer_config_bi_multi
                       )
predictor1.fit_model(
                     epochs = 7,
                     show_progress= 1,
                     validation_split = 0.20,
                     monitor='val_loss',
                     patience=1,
                     min_delta = 0,
                     verbose = 1
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(scaled_features[-5:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_bigru(epochs = 1)

In [ ]:
predictor1.predict(scaled_features[-5:])

In [ ]:
predictor1.freeze("test3.keras")

In [ ]:
load = PureMulti(steps_past =  5, steps_future = 10)

In [ ]:
load.set_model_id('Loaded Model')

In [ ]:
load.retrieve('test3.keras')

In [ ]:
load.predict(scaled_features[-5:])

In [ ]:
from imbrium import HybridMulti

In [ ]:
multi_hybrid_layer_config = {
            "layer0": {
                "config": {
                    "filters": 64,
                    "kernel_size": 1,
                    "activation": "relu",
                    "regularization": 0.0,
                    "dropout": 0.0,
                }
            },
            "layer1": {
                "config": {
                    "filters": 32,
                    "kernel_size": 1,
                    "activation": "relu",
                    "regularization": 0.0,
                    "dropout": 0.0,
                }
            },
            "layer2": {
                "config": {
                    "pool_size": 2,
                }
            },
            "layer3": {
                "config": {
                    "neurons": 32,
                    "activation": "relu",
                    "regularization": 0.0,
                    "dropout": 0.0,
                }
            },
            "layer4": {
                "config": {
                    "neurons": 32,
                    "activation": "relu",
                    "regularization": 0.0,
                }
            },
        }

In [ ]:
predictor1 = HybridMulti(sub_seq = 2, steps_past = 10, steps_future = 5,target = scaled_target, features = scaled_features)
predictor1.create_cnnrnn(optimizer= 'adam',
                         loss= 'mean_squared_error',
                         metrics = 'mean_squared_error',
                         layer_config = multi_hybrid_layer_config
                        )
predictor1.fit_model(
                     epochs = 3,
                     show_progress= 1,
                     validation_split = 0.20,
                     monitor='val_loss',
                     patience=1,
                     min_delta = 0,
                     verbose = 1,
                     board = True
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(scaled_features[-10:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.create_fit_cnnrnn(epochs=1)

In [ ]:
predictor1 = HybridMulti(sub_seq = 2, steps_past = 10, steps_future = 5, target = scaled_target, features = scaled_features)

predictor1.create_cnnlstm(
                          optimizer= 'adam',
                          loss= 'mean_squared_error',
                          metrics = 'mean_squared_error',
                          layer_config = multi_hybrid_layer_config
                         )
predictor1.fit_model(
                     epochs = 2,
                     show_progress= 1,
                     validation_split = 0.20
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(scaled_features[-10:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_cnnlstm(epochs=1)

In [ ]:
predictor1 = HybridMulti(sub_seq = 2, steps_past = 10, steps_future = 15, target = scaled_target, features = scaled_features)

predictor1.create_cnngru(
                         optimizer= 'adam',
                         loss= 'mean_squared_error',
                         metrics = 'mean_squared_error',
                         layer_config = multi_hybrid_layer_config
                        )
predictor1.fit_model(
                     epochs = 10,
                     show_progress= 1,
                     validation_split = 0.20
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(scaled_features[-10:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_cnngru(epochs=1)

In [ ]:
predictor1 = HybridMulti(sub_seq = 2, steps_past = 10, steps_future = 5, target = scaled_target, features = scaled_features)

predictor1.create_cnnbirnn(optimizer= 'adam',
                           loss= 'mean_squared_error',
                           metrics = 'mean_squared_error',
                           layer_config = multi_hybrid_layer_config)
predictor1.fit_model(
                     epochs = 2,
                     show_progress= 1,
                     validation_split = 0.20
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(scaled_features[-10:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_cnnbirnn(epochs=1)

In [ ]:
predictor1 = HybridMulti(sub_seq = 2, steps_past = 10, steps_future = 5, target = scaled_target, features = scaled_features)

predictor1.create_cnnbilstm(
                            optimizer= 'adam',
                            loss= 'mean_squared_error',
                            metrics = 'mean_squared_error',
                            layer_config = multi_hybrid_layer_config
                            )
predictor1.fit_model(
                     epochs = 2,
                     show_progress= 1,
                     validation_split = 0.20
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(scaled_features[-10:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_cnnbilstm(epochs=1)

In [ ]:
predictor1 = HybridMulti(sub_seq = 2, steps_past = 10, steps_future = 5, target = scaled_target, features = scaled_features)

predictor1.create_cnnbigru(optimizer= 'adam',
                           loss= 'mean_squared_error',
                           metrics = 'mean_squared_error',
                           layer_config = multi_hybrid_layer_config
                          )
predictor1.fit_model(epochs = 7,
                     show_progress= 1,
                     validation_split = 0.20,
                     monitor='val_loss',
                     patience=1,
                     min_delta = 0,
                     verbose = 1
                    )

In [ ]:
predictor1.show_performance()

In [ ]:
predictor1.predict(scaled_features[-10:])

In [ ]:
predictor1.model_blueprint()

In [ ]:
predictor1.create_fit_cnnbigru(epochs=1)

In [ ]:
predictor1.freeze("test4.keras")

In [ ]:
load = HybridMulti(sub_seq = 2, steps_past =  10, steps_future = 5)

In [ ]:
load.set_model_id('Loaded Model')

In [ ]:
load.retrieve('test4.keras')

In [ ]:
predictor1.predict(scaled_features[-10:])